In [1]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset
from pathlib import Path
import pandas as pd
from pprint import pprint

/home/chuongdao/workspace/robot_learning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#DATASET_ROOT= Path("/Users/chuongdao/workspace/robotic_foundations/robot_learning/robot_learning/dataset")
DATASET_ROOT= Path("/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam")

In [5]:
DATASET_PATH = DATASET_ROOT
DATASET_DATA_CHUNK_PARQUET_PATH = DATASET_PATH / "data" / "chunk-000" / "file-000.parquet"
DATASET_VIDEOS_PATH = DATASET_PATH / "videos"
DATASET_STATS_PATH = DATASET_PATH /  "meta/stats.json"
INFO_STATS_PATH = DATASET_PATH /  "meta/info.json"
DATASET_TASKS_PARQUET_PATH = DATASET_PATH / "meta" / "tasks.parquet"
DATASET_META_CHUNK_PARQUET_PATH = DATASET_PATH / "meta" / "episodes" / "chunk-000" / "file-000.parquet"

In [6]:
!tree -t $DATASET_PATH

/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam
├── data
│   └── chunk-000
│       ├── file-000.parquet
│       ├── file-001.parquet
│       ├── file-002.parquet
│       ├── file-003.parquet
│       ├── file-004.parquet
│       ├── file-005.parquet
│       ├── file-006.parquet
│       ├── file-007.parquet
│       ├── file-008.parquet
│       ├── file-009.parquet
│       ├── file-010.parquet
│       ├── file-011.parquet
│       └── file-012.parquet
├── videos
│   ├── observation.images.scene
│   │   └── chunk-000
│   │       ├── file-000.mp4
│   │       ├── file-001.mp4
│   │       ├── file-002.mp4
│   │       ├── file-003.mp4
│   │       ├── file-004.mp4
│   │       ├── file-005.mp4
│   │       ├── file-006.mp4
│   │       ├── file-007.mp4
│   │       ├── file-008.mp4
│   │       ├── file-009.mp4
│   │       ├── file-010.mp4
│   │       ├── file-011.mp4
│   │       ├── file-012.mp4
│   │       ├── file-013.mp4
│   │       └── file-014.mp4
│   └── observa

## EXPLORE DATASET

#### DATA PARQUET

In [9]:
data_df = pd.read_parquet(DATASET_DATA_CHUNK_PARQUET_PATH)

In [14]:
pprint(data_df.columns.to_list())

['action',
 'observation.state',
 'timestamp',
 'frame_index',
 'episode_index',
 'index',
 'task_index']


In [15]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10346 entries, 0 to 10345
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   action             10346 non-null  object 
 1   observation.state  10346 non-null  object 
 2   timestamp          10346 non-null  float32
 3   frame_index        10346 non-null  int64  
 4   episode_index      10346 non-null  int64  
 5   index              10346 non-null  int64  
 6   task_index         10346 non-null  int64  
dtypes: float32(1), int64(4), object(2)
memory usage: 525.5+ KB


In [10]:
data_df.head()

,action,observation.state,timestamp,frame_index,episode_index,index,task_index
0,"[8.7378645, -99.14163, 99.783195, 47.351524, 0...","[8.918062, -98.62946, 99.011536, 47.07379, -0....",0.000000,0,0,0,0
1,"[8.7378645, -99.14163, 99.783195, 47.351524, 0...","[8.918062, -98.62946, 99.011536, 47.07379, -0....",0.033333,1,0,1,0
2,"[8.7378645, -99.14163, 99.783195, 47.351524, 0...","[8.918062, -98.62946, 99.011536, 47.07379, -0....",0.066667,2,0,2,0
3,"[8.7378645, -99.14163, 99.783195, 47.351524, 0...","[8.918062, -98.62946, 99.011536, 47.07379, -0....",0.100000,3,0,3,0
4,"[8.7378645, -99.14163, 99.783195, 47.351524, 0...","[8.918062, -98.62946, 99.011536, 47.07379, -0....",0.133333,4,0,4,0


In [18]:
data_df.describe()

,timestamp,frame_index,episode_index,index,task_index
count,10346.000000,10346.000000,10346.000000,10346.000000,10346.0
mean,17.226671,516.800116,4.499807,5172.500000,0.0
std,9.955926,298.677790,2.872521,2986.777277,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0
25%,8.600000,258.000000,2.000000,2586.250000,0.0
50%,17.233334,517.000000,4.500000,5172.500000,0.0
75%,25.833334,775.000000,7.000000,7758.750000,0.0
max,34.466667,1034.000000,9.000000,10345.000000,0.0


In [ ]:
# Define the new task and episodes to update
MARKER_TASK = "Pick the marker and place it in the box"
episodes_to_update = [63, 64, 65, 66, 67, 68, 69, 70, 71, 72]

# Step 1: Load tasks.parquet and add MARKER_TASK if it doesn't exist
tasks_df = pd.read_parquet(DATASET_TASKS_PARQUET_PATH)

# Check if MARKER_TASK already exists
if MARKER_TASK not in tasks_df.index:
    # Get the next task_index (highest existing + 1)
    next_task_index = tasks_df['task_index'].max() + 1 if len(tasks_df) > 0 else 0
    
    # Add new task to tasks_df
    tasks_df.loc[MARKER_TASK] = next_task_index
    print(f"✅ Added '{MARKER_TASK}' with task_index {next_task_index}")
    
    # Save updated tasks.parquet
    tasks_df.to_parquet(DATASET_TASKS_PARQUET_PATH, index=True)
    print(f"✅ Saved updated tasks.parquet to {DATASET_TASKS_PARQUET_PATH}")
else:
    next_task_index = tasks_df.loc[MARKER_TASK, 'task_index']
    print(f"ℹ️  Task '{MARKER_TASK}' already exists with task_index {next_task_index}")

# Display tasks
print("\nCurrent tasks:")
print(tasks_df)


Added 'Pick the marker and place it in the box' with task_index 1
Saved updated tasks.parquet to /home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/tasks.parquet

Current tasks:
                                             task_index
Pick the black cube and place it in the box           0
Pick the marker and place it in the box               1


In [34]:
# Step 2: Find all data parquet files and search for episodes 63-72
DATA_DIR = DATASET_PATH / "data" / "chunk-000"
data_parquet_files = sorted(DATA_DIR.glob("file-*.parquet"))

print(f"Found {len(data_parquet_files)} data parquet files in {DATA_DIR}")
print("\nScanning files for episodes 63-72...")

files_to_update = {}  # {file_path: {episodes: [list], min_ep: int, max_ep: int}}

for file_path in data_parquet_files:
    df = pd.read_parquet(file_path)
    episode_range = (df['episode_index'].min(), df['episode_index'].max())
    episodes_in_file = df['episode_index'].unique()
    
    # Check if any of our target episodes are in this file
    matching_episodes = [ep for ep in episodes_to_update if ep in episodes_in_file]
    
    if matching_episodes:
        files_to_update[file_path] = {
            'episodes': matching_episodes,
            'min_ep': episode_range[0],
            'max_ep': episode_range[1],
            'total_rows': len(df)
        }
        print(f"  📄 {file_path.name}: episodes {episode_range[0]}-{episode_range[1]} "
              f"→ contains {len(matching_episodes)} target episodes: {matching_episodes}")

print(f"\n✅ Found {len(files_to_update)} file(s) that need updating")


Found 13 data parquet files in /home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/data/chunk-000

Scanning files for episodes 63-72...
  📄 file-010.parquet: episodes 63-64 → contains 2 target episodes: [63, 64]
  📄 file-011.parquet: episodes 65-65 → contains 1 target episodes: [65]
  📄 file-012.parquet: episodes 66-72 → contains 7 target episodes: [66, 67, 68, 69, 70, 71, 72]

✅ Found 3 file(s) that need updating


In [35]:
# Step 3: Verify which episodes we found and check their current task_index
all_found_episodes = set()
for file_info in files_to_update.values():
    all_found_episodes.update(file_info['episodes'])

missing_episodes = set(episodes_to_update) - all_found_episodes
if missing_episodes:
    print(f"⚠️  Warning: Episodes {sorted(missing_episodes)} not found in any data file!")
else:
    print(f"✅ All episodes {episodes_to_update} found in data files")

print(f"\nEpisodes found: {sorted(all_found_episodes)}")

# Check current task_index for these episodes
print("\nCurrent task_index values for these episodes:")
for file_path, info in files_to_update.items():
    df = pd.read_parquet(file_path)
    mask = df['episode_index'].isin(info['episodes'])
    current_task_indices = df[mask]['task_index'].unique()
    print(f"  {file_path.name}: episodes {info['episodes']} → task_index {current_task_indices}")

✅ All episodes [63, 64, 65, 66, 67, 68, 69, 70, 71, 72] found in data files

Episodes found: [63, 64, 65, 66, 67, 68, 69, 70, 71, 72]

Current task_index values for these episodes:
  file-010.parquet: episodes [63, 64] → task_index [0]
  file-011.parquet: episodes [65] → task_index [0]
  file-012.parquet: episodes [66, 67, 68, 69, 70, 71, 72] → task_index [1]


In [ ]:
# Step 3: Verify which episodes we found and check their current task_index
all_found_episodes = set()
for file_info in files_to_update.values():
    all_found_episodes.update(file_info['episodes'])

missing_episodes = set(episodes_to_update) - all_found_episodes
if missing_episodes:
    print(f"⚠️  Warning: Episodes {sorted(missing_episodes)} not found in any data file!")
else:
    print(f"✅ All episodes {episodes_to_update} found in data files")

print(f"\nEpisodes found: {sorted(all_found_episodes)}")

# Check current task_index for these episodes
print("\nCurrent task_index values for these episodes:")
for file_path, info in files_to_update.items():
    df = pd.read_parquet(file_path)
    mask = df['episode_index'].isin(info['episodes'])
    current_task_indices = df[mask]['task_index'].unique()
    print(f"  {file_path.name}: episodes {info['episodes']} → task_index {current_task_indices}")

Found 8273 rows from episodes [63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
Current task_index values: [0]

Sample of rows to update:
   episode_index  frame_index  task_index
0             66            0           0
1             66            1           0
2             66            2           0
3             66            3           0
4             66            4           0
5             66            5           0
6             66            6           0
7             66            7           0
8             66            8           0
9             66            9           0


In [36]:
# Step 4: Update task_index for episodes 63-72 in all relevant files
total_rows_updated = 0

for file_path, info in files_to_update.items():
    print(f"\n📝 Processing {file_path.name}...")
    
    # Load the file
    df = pd.read_parquet(file_path)
    
    # Create mask for episodes in this file
    mask = df['episode_index'].isin(info['episodes'])
    rows_to_update = df[mask]
    
    print(f"  Found {len(rows_to_update)} rows from episodes {info['episodes']}")
    print(f"  Current task_index: {rows_to_update['task_index'].unique()}")
    
    # Update task_index
    df.loc[mask, 'task_index'] = next_task_index
    
    # Verify
    updated_rows = df[mask]
    print(f"  ✅ Updated to task_index {next_task_index}")
    print(f"  Verification: All rows have correct task_index? {(updated_rows['task_index'] == next_task_index).all()}")
    
    # Save the file
    df.to_parquet(file_path, index=False)
    print(f"  💾 Saved {file_path.name}")
    
    total_rows_updated += len(rows_to_update)

print(f"\n{'='*60}")
print(f"✅ Summary: Updated {total_rows_updated} total rows across {len(files_to_update)} file(s)")
print(f"✅ All episodes {episodes_to_update} now have task_index {next_task_index} ('{MARKER_TASK}')")
print(f"{'='*60}")



📝 Processing file-010.parquet...
  Found 2363 rows from episodes [63, 64]
  Current task_index: [0]
  ✅ Updated to task_index 1
  Verification: All rows have correct task_index? True
  💾 Saved file-010.parquet

📝 Processing file-011.parquet...
  Found 1182 rows from episodes [65]
  Current task_index: [0]
  ✅ Updated to task_index 1
  Verification: All rows have correct task_index? True
  💾 Saved file-011.parquet

📝 Processing file-012.parquet...
  Found 8273 rows from episodes [66, 67, 68, 69, 70, 71, 72]
  Current task_index: [1]
  ✅ Updated to task_index 1
  Verification: All rows have correct task_index? True
  💾 Saved file-012.parquet

✅ Summary: Updated 11818 total rows across 3 file(s)
✅ All episodes [63, 64, 65, 66, 67, 68, 69, 70, 71, 72] now have task_index 1 ('Pick the marker and place it in the box')


In [ ]:
# Step 6: Update episode metadata files (meta/episodes/chunk-000/file-*.parquet)
# Find which episode metadata files contain episodes 63-72
EPISODES_DIR = DATASET_PATH / "meta" / "episodes" / "chunk-000"
episode_parquet_files = sorted(EPISODES_DIR.glob("file-*.parquet"))

print(f"Found {len(episode_parquet_files)} episode metadata files")
print("\nScanning episode metadata files for episodes 63-72...")

episode_files_to_update = {}  # {file_path: {episodes: [list]}}

for file_path in episode_parquet_files:
    df = pd.read_parquet(file_path)
    episodes_in_file = df['episode_index'].unique()
    
    # Check if any of our target episodes are in this file
    matching_episodes = [ep for ep in episodes_to_update if ep in episodes_in_file]
    
    if matching_episodes:
        episode_files_to_update[file_path] = {
            'episodes': matching_episodes
        }
        print(f"  📄 {file_path.name}: contains {len(matching_episodes)} target episodes: {matching_episodes}")

print(f"\n✅ Found {len(episode_files_to_update)} episode metadata file(s) that need updating")


In [ ]:
# Step 7: Update "tasks" column in episode metadata files
print("Updating episode metadata files...\n")

for file_path, info in episode_files_to_update.items():
    print(f"📝 Processing {file_path.name}...")
    
    # Load the episode metadata file
    ep_df = pd.read_parquet(file_path)
    
    # Filter for episodes 63-72 in this file
    mask = ep_df['episode_index'].isin(info['episodes'])
    episodes_to_update_in_file = ep_df[mask]
    
    print(f"  Found {len(episodes_to_update_in_file)} episodes: {info['episodes']}")
    
    # Show current tasks before update
    print(f"  Current tasks column (first few):")
    for idx, row in episodes_to_update_in_file.head(3).iterrows():
        print(f"    Episode {row['episode_index']}: {row['tasks']}")
    
    # Update the "tasks" column to contain MARKER_TASK in a list
    # The tasks column should be a list with the task string
    ep_df.loc[mask, 'tasks'] = ep_df.loc[mask, 'tasks'].apply(lambda x: [MARKER_TASK])
    
    # Verify the update
    updated_episodes = ep_df[mask]
    print(f"  ✅ Updated tasks column")
    
    # Show updated tasks
    print(f"  New tasks column (first few):")
    for idx, row in updated_episodes.head(3).iterrows():
        print(f"    Episode {row['episode_index']}: {row['tasks']}")
    
    # Save the file
    ep_df.to_parquet(file_path, index=False)
    print(f"  💾 Saved {file_path.name}\n")

print(f"✅ Updated {len(episode_files_to_update)} episode metadata file(s)")


In [ ]:
# Step 8: Final verification for episode metadata
print("\n" + "="*60)
print("EPISODE METADATA VERIFICATION")
print("="*60)

# Reload and verify episode metadata files
for file_path, info in episode_files_to_update.items():
    df = pd.read_parquet(file_path)
    mask = df['episode_index'].isin(info['episodes'])
    updated_episodes = df[mask]
    
    print(f"\n{file_path.name}:")
    print(f"  Episodes: {info['episodes']}")
    print(f"  Rows: {len(updated_episodes)}")
    
    # Check that all tasks are correct
    all_correct = True
    for idx, row in updated_episodes.iterrows():
        if row['tasks'] != [MARKER_TASK]:
            print(f"    ❌ Episode {row['episode_index']}: tasks = {row['tasks']} (expected [{MARKER_TASK}])")
            all_correct = False
    
    if all_correct:
        print(f"  ✅ All episodes have correct tasks: [{MARKER_TASK}]")
    else:
        print(f"  ❌ Some episodes have incorrect tasks!")

print("\n" + "="*60)
print("COMPLETE SUMMARY")
print("="*60)
print(f"✅ Added '{MARKER_TASK}' to tasks.parquet with task_index {next_task_index}")
print(f"✅ Updated data files: {len(files_to_update)} file(s) with {total_rows_updated} rows")
print(f"✅ Updated episode metadata files: {len(episode_files_to_update)} file(s)")
print(f"✅ All episodes {episodes_to_update} now use '{MARKER_TASK}'")
print("="*60)


In [37]:
# Step 5: Verification - Check that updates were successful
print("\n" + "="*60)
print("FINAL VERIFICATION")
print("="*60)

# Reload files and verify
for file_path, info in files_to_update.items():
    df = pd.read_parquet(file_path)
    mask = df['episode_index'].isin(info['episodes'])
    updated_rows = df[mask]
    
    print(f"\n{file_path.name}:")
    print(f"  Episodes: {info['episodes']}")
    print(f"  Rows: {len(updated_rows)}")
    print(f"  Task index: {updated_rows['task_index'].unique()}")
    print(f"  All correct? {(updated_rows['task_index'] == next_task_index).all()} ✅")

print(f"\n✅ All files updated successfully!")
print(f"✅ Episodes {episodes_to_update} now use task_index {next_task_index} ('{MARKER_TASK}')")



FINAL VERIFICATION

file-010.parquet:
  Episodes: [63, 64]
  Rows: 2363
  Task index: [1]
  All correct? True ✅

file-011.parquet:
  Episodes: [65]
  Rows: 1182
  Task index: [1]
  All correct? True ✅

file-012.parquet:
  Episodes: [66, 67, 68, 69, 70, 71, 72]
  Rows: 8273
  Task index: [1]
  All correct? True ✅

✅ All files updated successfully!
✅ Episodes [63, 64, 65, 66, 67, 68, 69, 70, 71, 72] now use task_index 1 ('Pick the marker and place it in the box')


### TASKS PARQUET

In [7]:
tasks_df = pd.read_parquet(DATASET_TASKS_PARQUET_PATH)

In [8]:
tasks_df.head()

,task_index
Pick small black cube on table and put it in the box,0


In [20]:
dataset = LeRobotDataset(root=DATASET_PATH.as_posix(), 
                         repo_id="pick_small_cube", 
                         episodes=[1])

### DATA CHUNK PARQUET 

In [9]:
data_chunk_df = pd.read_parquet(DATASET_META_CHUNK_PARQUET_PATH)

In [10]:
data_chunk_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Columns: 107 entries, episode_index to meta/episodes/file_index
dtypes: float64(4), int64(12), object(91)
memory usage: 2.6+ KB


In [11]:
data_chunk_df.head()

,episode_index,tasks,length,data/chunk_index,data/file_index,dataset_from_index,dataset_to_index,videos/observation.images.scene/chunk_index,videos/observation.images.scene/file_index,videos/observation.images.scene/from_timestamp,...,stats/task_index/mean,stats/task_index/std,stats/task_index/count,stats/task_index/q01,stats/task_index/q10,stats/task_index/q50,stats/task_index/q90,stats/task_index/q99,meta/episodes/chunk_index,meta/episodes/file_index
0,0,[Pick the black cube and place it in the box],1182,0,0,0,1182,0,0,0.000000,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,0
1,1,[Pick the black cube and place it in the box],1181,0,0,1182,2363,0,0,39.400000,...,[0.0],[0.0],[1181],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.599999999999476e-14],[3.9599999999994235e-14],0,0
2,2,[Pick the black cube and place it in the box],1182,0,0,2363,3545,0,0,78.766667,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,0


## EDIT PARQUET TASKS DATA

In [8]:
tasks_df = pd.read_parquet(DATASET_TASKS_PARQUET_PATH)
tasks_df.head()

,task_index
Pick the black cube and place it in the box,0


In [14]:
episode_parquet_files = list(DATASET_PATH.glob("meta/episodes/chunk-*/*.parquet"))

In [16]:
episode_parquet_files = sorted(episode_parquet_files)

In [17]:
episode_parquet_files

[PosixPath('/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/episodes/chunk-000/file-000.parquet'),
 PosixPath('/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/episodes/chunk-000/file-001.parquet'),
 PosixPath('/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/episodes/chunk-000/file-002.parquet'),
 PosixPath('/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/episodes/chunk-000/file-003.parquet'),
 PosixPath('/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/episodes/chunk-000/file-004.parquet'),
 PosixPath('/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/episodes/chunk-000/file-005.parquet'),
 PosixPath('/home/chuongdao/.cache/huggingface/lerobot/dataset/pick_and_place_realsense_cam/meta/episodes/chunk-000/file-006.parquet'),
 PosixPath('/home/chuongdao/.cache/huggingface/l

In [23]:
ep12_df = pd.read_parquet(episode_parquet_files[12])

In [25]:
ep12_df.head(50)

,episode_index,tasks,length,data/chunk_index,data/file_index,dataset_from_index,dataset_to_index,videos/observation.images.scene/chunk_index,videos/observation.images.scene/file_index,videos/observation.images.scene/from_timestamp,...,stats/task_index/mean,stats/task_index/std,stats/task_index/count,stats/task_index/q01,stats/task_index/q10,stats/task_index/q50,stats/task_index/q90,stats/task_index/q99,meta/episodes/chunk_index,meta/episodes/file_index
0,66,[Pick the black cube and place it in the box],1182,0,12,77996,79178,0,14,0.0,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,12
1,67,[Pick the black cube and place it in the box],1182,0,12,79178,80360,0,14,39.4,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,12
2,68,[Pick the black cube and place it in the box],1182,0,12,80360,81542,0,14,78.8,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,12
3,69,[Pick the black cube and place it in the box],1182,0,12,81542,82724,0,14,118.2,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,12
4,70,[Pick the black cube and place it in the box],1182,0,12,82724,83906,0,14,157.6,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,12
5,71,[Pick the black cube and place it in the box],1182,0,12,83906,85088,0,14,197.0,...,[0.0],[0.0],[1182],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.5999999999994754e-14],[3.9599999999994235e-14],0,12
6,72,[Pick the black cube and place it in the box],1181,0,12,85088,86269,0,14,236.4,...,[0.0],[0.0],[1181],[3.9999999999994176e-16],[3.999999999999417e-15],[1.9999999999997088e-14],[3.599999999999476e-14],[3.9599999999994235e-14],0,12


In [26]:
MARKER_TASK = "Pick the marker and place it in the box"